In [ ]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP, ss_to_index

import numpy as np
import torch
from torch_geometric.data import Data

In [ ]:
dist_th = 12.0  # threshold distance between CA atoms below which edges will be built
idx_mode = 60  # first mode = 0, last mode = 63

np.random.seed(0)
torch.manual_seed(0)

task_name_temp = 'all_th'+str(int(dist_th))
task_name = task_name_temp+'_mode'+str(idx_mode)

In [ ]:
with open("data/all_proteins_freqs_no_ratios.dat") as f:
    lines = f.read().split("\n")

N_seq = len(lines)-1 # ignore the last blank line

pdb_ids = []
freqs_mode0 = []
freqs = []
for idx in range(N_seq):
    line = lines[idx].split(" ")
    pdb_ids.append(line[1])
    freqs_mode0.append(float(line[2]))
    freqs.append(float(line[idx_mode+2]))

In [ ]:
data_list_mode0 = torch.load('data/'+task_name_temp+'_mode0.pt')

In [ ]:
data_list = []

idx_freqs = 0
for idx, data_mode0 in enumerate(data_list_mode0):
    while data_mode0.y != freqs_mode0[idx_freqs]:
        idx_freqs += 1  # skip the indices of excluded_pdb_ids
    
    # replace the first mode with another mode
    data = Data(edge_attr=data_mode0.edge_attr, 
                edge_index=data_mode0.edge_index, 
                x=data_mode0.x, 
                y=torch.tensor([freqs[idx_freqs]], dtype=torch.float))
    data_list.append(data)

In [ ]:
N_data = len(data_list)
print(f'Number of proteins: {N_data}')

In [ ]:
torch.save(data_list, 'data/'+task_name+'.pt')